In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

Load horses vs humans dataset from tensor flow datasets

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'horses_or_humans',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

Create and train the model - MLP

In [ ]:
for img, label in ds_train.take(1):
  print(img.shape, label)

(128, 300, 300, 3) tf.Tensor(
[0 0 1 0 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 0 0 1 0 1 1 1 1 0 0 1 1 0 0 0 0
 0 0 0 1 0 0 0 1 1 1 1 1 0 1 0 1 1 0 0 1 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0
 0 0 1 1 1 1 1 0 1 0 1 0 0 0 1 1 0 1 0 1 1 0 0 0 1 0 0 1 1 1 0 1 1 1 1 0 1
 1 1 1 0 0 0 0 1 1 1 1 0 0 1 1 0 0], shape=(128,), dtype=int64)


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(300, 300, 3)),
  tf.keras.layers.Dense(128,activation='relu'),
  tf.keras.layers.Dense(64,activation='relu'),
  tf.keras.layers.Dense(64,activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.Adam(0.001),
    metrics=['accuracy'],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
9/9 [==============================] - 4s 443ms/step - loss: 20.2585 - accuracy: 0.4927 - val_loss: 17.9761 - val_accuracy: 0.5000
Epoch 2/6
9/9 [==============================] - 4s 422ms/step - loss: 4.8819 - accuracy: 0.5346 - val_loss: 8.1788 - val_accuracy: 0.5000
Epoch 3/6
9/9 [==============================] - 4s 431ms/step - loss: 3.1743 - accuracy: 0.5648 - val_loss: 25.8976 - val_accuracy: 0.5000
Epoch 4/6
9/9 [==============================] - 4s 462ms/step - loss: 10.7324 - accuracy: 0.5102 - val_loss: 7.8958 - val_accuracy: 0.5039
Epoch 5/6
9/9 [==============================] - 4s 426ms/step - loss: 2.5360 - accuracy: 0.5949 - val_loss: 1.4670 - val_accuracy: 0.7891
Epoch 6/6
9/9 [==============================] - 4s 437ms/step - loss: 0.9297 - accuracy: 0.7838 - val_loss: 2.4329 - val_accuracy: 0.7031
